In [1]:
import pandas as pd

In [2]:
import pickle

In [3]:
import seaborn as sns
import matplotlib.pyplot as plt

In [4]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error

In [6]:
df = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet')

In [ ]:
Question 1. Downloading the data

In [7]:
print("Columns Names:", df.columns)           
print("# columns:", len(df.columns))  

Columns Names: Index(['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'trip_distance', 'RatecodeID', 'store_and_fwd_flag',
       'PULocationID', 'DOLocationID', 'payment_type', 'fare_amount', 'extra',
       'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge',
       'total_amount', 'congestion_surcharge', 'airport_fee'],
      dtype='object')
# columns: 19


In [ ]:
Question 2. Computing duration

In [ ]:

df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)
std_dev = df['duration'].std()
print(std_dev)


In [ ]:
beforeamount=df.shape[0]
print("Amount of data before:", beforeamount)           

In [ ]:
df = df[(df.duration >= 1) & (df.duration <= 60)]
afteramount=df.shape[0]
print("Amount of data after:", afteramount)   



In [ ]:
Question 3. Dropping outliers

In [ ]:
print("Dropping outliers:", afteramount*100/beforeamount)           

In [ ]:
categorical = ['PULocationID', 'DOLocationID']
numerical = ['trip_distance']

df[categorical] = df[categorical].astype(str)

In [ ]:
train_dicts = df[categorical + numerical].to_dict(orient='records')

dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)

target = 'duration'
y_train = df[target].values

lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)

mean_squared_error(y_train, y_pred)

In [ ]:
sns.distplot(y_pred, label='prediction')
sns.distplot(y_train, label='actual')

plt.legend()

In [5]:
def read_dataframe(filename):
    if filename.endswith('.csv'):
        dft = pd.read_csv(filename)

        dft.tpep_dropoff_datetime = pd.to_datetime(df.tpep_dropoff_datetime)
        dft.tpep_pickup_datetime = pd.to_datetime(df.tpep_pickup_datetime)
    elif filename.endswith('.parquet'):
        dft = pd.read_parquet(filename)

    dft['duration'] = dft.tpep_dropoff_datetime - dft.tpep_pickup_datetime
    dft.duration = dft.duration.apply(lambda td: td.total_seconds() / 60)

    dft = dft[(dft.duration >= 1) & (dft.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    dft[categorical] = dft[categorical].astype(str)
    
    return dft

In [6]:
df_train = read_dataframe('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet')
df_val = read_dataframe('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet')

In [8]:
len(df_train), len(df_val)

(3009173, 2855951)

In [7]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [7]:
categorical = ['PULocationID', 'DOLocationID'] #'PU_DO']#,
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)



In [ ]:
Question 4. One-hot encoding

In [18]:
print("number of columns:", X_train.shape)

number of columns: (3009173, 515)


In [8]:
val_dicts = df_val[categorical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [9]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [ ]:
Question 5. Training a model. RMSE on train

In [12]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)

rmse = mean_squared_error(y_train, y_pred, squared=False)

print(f"RMSE: {rmse:.2f} minutes")

RMSE: 7.65 minutos


In [ ]:
Question 6. Evaluating the model. RMSE on validation

In [13]:
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

rmse = mean_squared_error(y_val, y_pred, squared=False)

print(f"RMSE: {rmse:.2f} minutes")

RMSE: 7.81 minutos
